### Классификация текста

Одной из широко используемых задач NLP в различных бизнес-задачах является “классификация текста”. Цель классификации текста состоит в том, чтобы автоматически классифицировать текстовые документы по одной или нескольким определенным категориям. Некоторые примеры классификации текста:
- Определенеие настроений аудитории из социальных сетей
- Обнаружение спама и нежелательных писем
- Автоматическая маркировка запросов клиентов
- Категоризация новостных статей по определенным темам

Классификация текста является примером задачи машинного обучения с учителем, поскольку для обучения классификатора используется помеченный набор данных, содержащий текстовые документы и их метки. Пайплайн классификации текста состоит из 4 основных частей:
1. Подготовка данных
2. Генерация признаков
3. Тренировка модели
4. Оценка и тьюнинг модели

Проделаем данные шаги.

In [2]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re
import pandas, numpy, textblob, string
#!pip install wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Professional\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Professional\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Professional\AppData\Roaming\nltk_data...


In [3]:
import warnings
warnings.filterwarnings("ignore")

#### Подготовка датасета

В качестве датасета возьмем набор данных Amazon reviews. Набор данных состоит из 3,6 млн текстовых обзоров и их меток, мы будем использовать только небольшую часть данных. Датасет содержит 'text' текст отзыва и 'label' метку.

In [6]:
# Загружаем данные
data = open('datasets/corpus_HW6').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# создаем df
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [7]:
trainDF.head(5)

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2


#### Предобработка текста

Сделаем некоторую базовую предобработку текста: 
- уберем апострофы
- уберем специальные символы
- приведем к нижнему регистру
- сделаем токенизацию
- уберем стоп-слова 
- сделаем лемматизацию

In [8]:
def removeApostrophe(review):
    review = re.sub(r"won't", "will not", review)
    review = re.sub(r"can\'t", "can not", review)
    review = re.sub(r"n\'t", " not", review)
    review = re.sub(r"\'re", " are", review)
    review = re.sub(r"\'s", " is", review)
    review = re.sub(r"\'d", " would", review)
    review = re.sub(r"\'ll", " will", review)
    review = re.sub(r"\'t", " not", review)
    review = re.sub(r"\'ve", " have", review)
    review = re.sub(r"\'m", " am", review)
    return review

def removeApostropheFixed(review):
    review = re.sub(r"won't", "willnot", review)
    review = re.sub(r"can\'t", "cannot", review)
    review = re.sub(r"\snot", "not", review)
    review = re.sub(r"n\'t", "not", review)
    review = re.sub(r"\'re", " are", review)
    review = re.sub(r"\'s", " is", review)
    review = re.sub(r"\'d", " would", review)
    review = re.sub(r"\'ll", " will", review)
    review = re.sub(r"\'t", " not", review)
    review = re.sub(r"\'ve", " have", review)
    review = re.sub(r"\'m", " am", review)
    return review

def removeSpecialChars(review):
     return re.sub('[^a-zA-Z]', ' ', review)
    
def doCleaningFixed(review):
    review = removeApostropheFixed(review)
    review = removeSpecialChars(review) 
    review = review.lower()  
    review = review.split() #Tokenization
    lmtzr = WordNetLemmatizer()
    review = [lmtzr.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = " ".join(review)    
    return review

def doTextCleaning(review):
    review = removeApostrophe(review)
    review = removeSpecialChars(review) 
    review = review.lower()  
    review = review.split() #Tokenization
    lmtzr = WordNetLemmatizer()
    review = [lmtzr.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = " ".join(review)    
    return review

In [9]:
text = "I won't fdfd \'m  ase\'re can\'t this won't"

In [10]:
removeApostrophe(text)

'I will not fdfd  am  ase are can not this will not'

In [11]:
trainDF['textOld'] = trainDF['text'].apply(doTextCleaning)
trainDF['textNew'] = trainDF['text'].apply(doCleaningFixed)

In [12]:
trainDF['textOld'].head(20)

0     stuning even non gamer sound track beautiful p...
1     best soundtrack ever anything reading lot revi...
2     amazing soundtrack favorite music time hand in...
3     excellent soundtrack truly like soundtrack enj...
4     remember pull jaw floor hearing played game kn...
5     absolute masterpiece quite sure actually takin...
6     buyer beware self published book want know rea...
7     glorious story loved whisper wicked saint stor...
8     five star book finished reading whisper wicked...
9     whisper wicked saint easy read book made want ...
10    worst complete waste time typographical error ...
11    great book great book could put could read fas...
12    great read thought book brilliant yet realisti...
13    oh please guess romance novel lover one discer...
14    awful beyond belief feel write keep others was...
15    try fool u fake review glaringly obvious glowi...
16    romantic zen baseball comedy hear folk say mak...
17    fashionable compression stocking dvt docto

In [13]:
trainDF['textNew'].head(20)

0     stuning even non gamer sound track beautiful p...
1     best soundtrack ever anything reading lot revi...
2     amazing soundtrack favorite music time hand in...
3     excellent soundtrack truly like soundtrack enj...
4     remember pull jaw floor hearing played game kn...
5     absolute masterpiece quite sure actually takin...
6     buyer beware self published book want know rea...
7     glorious story loved whisper wicked saint stor...
8     five star book finished reading whisper wicked...
9     whisper wicked saint easy read book made want ...
10    worst complete waste time typographical error ...
11    great book great book couldnot put couldnot re...
12    great read thought book brilliant yet realisti...
13    oh please guess romance novel lover one andnot...
14    awful beyond belief feel write keep others was...
15    donot try fool u fake review glaringly obvious...
16    romantic zen baseball comedy hear folk say don...
17    fashionable compression stocking dvt docto

In [14]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['textOld'], trainDF['label'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

#### Feature Engineering

На этом этапе необработанные текстовые данные будут преобразованы в векторы признаков, а новые признаки будут созданы с использованием существующего набора данных. Мы реализуем следующие идеи, чтобы получить релевантные признаки из нашего набора данных:

- Count вектора как признаки
- TF-IDF вектора как признаки
  - Word level
  - N-Gram level
  - Character level
- Text / NLP признаки

##### Count Vectors
Count Vector - это матричная нотация набора данных, в которой каждая строка представляет документ из корпуса, каждый столбец представляет термин из корпуса, а каждая ячейка представляет счетчик частот конкретного термина в конкретном документе.

In [15]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['textOld'])

xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

##### TF-IDF вектора

Оценка TF-IDF представляет собой относительную важность термина в документе и во всем корпусе. 

TF (t) = (количество раз, когда термин t появляется в документе) / (общее количество терминов в документе)
IDF (t) = log_e(общее количество документов / количество документов с термином t в нем).

Векторы TF-IDF могут быть сгенерированы на разных уровнях входных токенов (слова, символы, n-граммы).

- Word Level TF-IDF : матрица содержит tf-idf оценки каждого слова в разных документах 
- N-gram Level TF-IDF : N-грам - комбинация N слов вместе. Матрица содержит tf-idf оценки N-грамов.
- Character Level TF-IDF : матрица содержит tf-idf оценки N-грам символов.

In [16]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['textOld'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['textOld'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['textOld'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [17]:
xtrain_tfidf_ngram_chars.todense()

matrix([[0.05037352, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.02669317, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.02994059, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.12653047, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.01814534, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [18]:
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))

In [19]:
trainDF.groupby("label")['word_count'].median()

label
__label__1    74.0
__label__2    68.0
Name: word_count, dtype: float64

##### Text / NLP признаки

Можно создать ряд дополнительных текстовых функций, которые иногда полезны для улучшения моделей классификации текста. Вот некоторые примеры:

1. Word Count документов – Общее количество слов в документе
2. Character Count документов – общее количество символов в документе
3. Average Word Density документов – средняя длина слов в документе
4. Puncutation Count – общее количество символов пунктуции в документе
5. Upper Case Count – общее количество слов с верхним регистром в документе
6. Title Word Count – общее количество заголовков  в документе
7. Частотное распределение тэгов частей речи:
  - Noun Count - количество существительных
  - Verb Count - количество глаголов
  - Adjective Count - количество прилагательных
  - Adverb Count - количество наречий
  - Pronoun Count - количество местоимений
  
Эти признаки довольно экспериментальны.

In [20]:
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [21]:
trainDF[['char_count', 'word_count', 'word_density', 'punctuation_count', 'title_word_count', 'upper_case_word_count' ]]

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count
0,426,80,5.259259,11,10,3
1,509,97,5.193878,14,7,3
2,760,129,5.846154,40,24,4
3,743,118,6.243697,33,52,4
4,481,87,5.465909,22,30,0
...,...,...,...,...,...,...
9995,867,152,5.666667,25,14,3
9996,861,141,6.063380,14,16,0
9997,650,108,5.963303,17,11,0
9998,135,27,4.821429,6,2,1


In [22]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# подсчет тэгов частей речи в предложении
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

trainDF['noun_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDF['verb_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDF['adj_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adj'))
trainDF['adv_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adv'))
trainDF['pron_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'pron'))

In [23]:
trainDF[['noun_count', 'verb_count', 'adj_count', 'adv_count', 'pron_count']]

,noun_count,verb_count,adj_count,adv_count,pron_count
0,20,15,6,6,11
1,20,23,9,3,10
2,39,18,13,10,11
3,52,12,9,2,7
4,31,13,7,2,9
...,...,...,...,...,...
9995,46,24,13,10,5
9996,49,15,14,5,3
9997,30,18,14,2,8
9998,3,10,1,4,4


#### Построение моделей

Последний шаг в процессе классификации текста - это обучение классификатора с использованием признаков, созданных на предыдущем шаге. Опробуем несколько вариантов моделей машинного обучения.

In [24]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    classifier.fit(feature_vector_train, label)
    
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

##### Naive Bayes

Наивный Байес - это метод классификации, основанный на теореме Байеса с допущением независимости между предикторами. Наивный байесовский классификатор предполагает, что наличие определенного признака в классе не связано с наличием какого-либо другого признака в нем.

In [25]:
# Naive Bayes на Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes на Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes на Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes на Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)


NB, Count Vectors:  0.8424
NB, WordLevel TF-IDF:  0.8428
NB, N-Gram Vectors:  0.7816
NB, CharLevel Vectors:  0.8052


In [26]:
classifier = naive_bayes.MultinomialNB()
classifier.fit(xtrain_tfidf_ngram_chars, train_y)    
predictions = classifier.predict(xvalid_tfidf_ngram_chars)
predictions

array([0, 0, 1, ..., 0, 0, 0])

 ##### Linear Classifier (Логистическая регрессия)
 
Логистическая регрессия измеряет связь между категориальной зависимой переменной и одной или несколькими независимыми переменными путем оценки вероятностей, используя логистическую/сигмоидную функцию. 

In [27]:
trainDF['label'].value_counts()

__label__1    5097
__label__2    4903
Name: label, dtype: int64

In [28]:
# Linear Classifier на Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: ", accuracy)

# Linear Classifier на Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier на Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier на Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)


LR, Count Vectors:  0.8516
LR, WordLevel TF-IDF:  0.8528
LR, N-Gram Vectors:  0.776
LR, CharLevel Vectors:  0.8196


##### SVM

SVM -  алгоритм машинного обучения с учителем, который может быть использован как для классификации, так и для регрессии. Модель извлекает наилучшую возможную гиперплоскость/линию, которая разделяет эти два класса.

In [29]:
# SVM на Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)


SVM, N-Gram Vectors:  0.7716


##### RandomForest

Cлучайный лес - ансамбль решающих деревьев.

In [30]:
from tqdm import tqdm_notebook

In [31]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [32]:
# RF на Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("RF, Count Vectors: ", accuracy)

# RF на Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: ", accuracy)


RF, Count Vectors:  0.8396
RF, WordLevel TF-IDF:  0.8276


##### Boosting Model

Бустинг - процедура последовательного построения композиции алгоритмов машинного обучения, когда каждый следующий алгоритм стремится компенсировать недостатки композиции всех предыдущих алгоритмов.

In [33]:
import xgboost

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# Extereme Gradient Boosting на Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print("Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting на Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting на Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: ", accuracy)


### Анализ тональности

Другая задача NLP - анализ тональности текста. Анализ тональности - определение полярности эмоциональных оценок в исследуемом тексте, который содержит мнения, суждения, эмоции, отношение автора к сущностям, личностям, вопросам, событиям, темам и их атрибутам.

С помощью SentimentIntensityAnalyzer определим тональность отзывов.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
sent_analysis = trainDF['text'].apply(sid.polarity_scores)
sent_analysis

In [ ]:
print(trainDF['text'][0])
print(sent_analysis[0])


In [ ]:
print(trainDF['text'][10])
print(sent_analysis[10])


#### Data Visualization

В заключении отобразим 
- самые популярные слова всех отзывов
- самые популярные слова положительных отзывов
- самые популярные слова отрицательных отзывов
в видео облака слов с помощью WordCloud.

In [ ]:
import collections

words = []
for line in trainDF['text']: 
    line_words = line.split()
    words.extend(line_words)

wordfreq = collections.Counter(words)
wordcloud = WordCloud(
 background_color='white',
 max_words=2000,
 stopwords=stopwords
 ).generate_from_frequencies(wordfreq)
plt.figure(figsize=(10,9))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
pos_words = []
for line in trainDF['text'][trainDF['label']=='__label__2']: 
    line_words = line.split()
    pos_words.extend(line_words)
pos_words = ' '.join(pos_words)

stopwords_set = set(stopwords.words('english'))
# draw a Word Cloud with word frequencies
wordcloud = WordCloud(
    background_color='white',
    max_words=50000,
    stopwords = stopwords_set
   ).generate(pos_words)
plt.figure(figsize=(13,12))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
negative = [r for r in trainDF['text'][trainDF['label']=='__label__1']]
neg = ''.join(negative)
# draw a Word Cloud with word frequencies
stopwords_set = set(stopwords.words('english'))
wordcloud = WordCloud(
    background_color='black',
    max_words=50000,
    stopwords = stopwords_set
   ).generate(neg)
plt.figure(figsize=(13,12))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Попробуем на русском

In [ ]:
import pandas as pd
data = pd.read_excel("datasets/reviews_for_the_summer.xls")

In [ ]:
data.head()

In [ ]:

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation

exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = (data['Rating'] > 3)*1

In [ ]:
data['target'] = data['target'].astype(int)
data.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
data['target'].value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

In [ ]:
count_vect = HashingVectorizer(n_features=200).fit(X_train.values)

In [ ]:
xtrain = count_vect.transform(X_train)
xtest = count_vect.transform(X_test)

In [ ]:
xtrain

In [ ]:
lr = linear_model.LogisticRegression(class_weight="balanced").fit(xtrain, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_test, lr.predict_proba(xtest)[:, 1])